In [1]:
import networkx as nx
import random
import csv
import math
import plotly.graph_objects as go
from io import StringIO

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat/2)**2 +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon/2)**2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def create_water_network(csv_data):
    points = []
    reader = csv.DictReader(StringIO(csv_data))
    for row in reader:
        try:
            if row['id'] and row['id'].isdigit():
                points.append({
                    'id': row['id'],
                    'x': float(row['Pressure 1']),
                    'y': float(row['Pressure 2'])
                })
        except (ValueError, KeyError):
            continue

    if not points:
        raise ValueError("No valid location data found in CSV")

    all_x = [p['x'] for p in points]
    all_y = [p['y'] for p in points]
    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)

    for p in points:
        p['x'] = 100 * (p['x'] - min_x) / (max_x - min_x) if max_x - min_x else 50
        p['y'] = 100 * (p['y'] - min_y) / (max_y - min_y) if max_y - min_y else 50

    num_reservoirs = min(2, len(points))
    num_consumers = min(5, len(points))
    num_junctions = min(10, max(0, len(points) - num_reservoirs - num_consumers))

    reservoir_points = points[:num_reservoirs]
    consumer_points = points[-num_consumers:]
    junction_points = points[num_reservoirs:num_reservoirs + num_junctions]

    G = nx.DiGraph()

    for point in reservoir_points:
        node_id = f"reservoir_{point['id']}"
        G.add_node(node_id, type="source", capacity=random.randint(800, 1200),
                   x=point['x'], y=point['y'])

    for point in junction_points:
        node_id = f"junction_{point['id']}"
        G.add_node(node_id, type="junction", x=point['x'], y=point['y'])

    for point in consumer_points:
        node_id = f"consumer_{point['id']}"
        G.add_node(node_id, type="sink", demand=random.randint(100, 300),
                   x=point['x'], y=point['y'])

    reservoirs = [n for n, a in G.nodes(data=True) if a['type'] == 'source']
    junctions = [n for n, a in G.nodes(data=True) if a['type'] == 'junction']
    consumers = [n for n, a in G.nodes(data=True) if a['type'] == 'sink']

    for reservoir in reservoirs:
        for junction in random.sample(junctions, random.randint(1, min(2, len(junctions)))):
            u_x, u_y = G.nodes[reservoir]['x'], G.nodes[reservoir]['y']
            v_x, v_y = G.nodes[junction]['x'], G.nodes[junction]['y']
            distance = math.sqrt((u_x - v_x)**2 + (u_y - v_y)**2)
            G.add_edge(reservoir, junction,
                       capacity=random.randint(300, 600),
                       cost=random.randint(5, 15),
                       distance=distance)

    for i, j1 in enumerate(junctions):
        for j2 in junctions[i + 1:]:
            if random.random() < 0.3:
                u_x, u_y = G.nodes[j1]['x'], G.nodes[j1]['y']
                v_x, v_y = G.nodes[j2]['x'], G.nodes[j2]['y']
                distance = math.sqrt((u_x - v_x)**2 + (u_y - v_y)**2)
                cap = random.randint(200, 400)
                cost = random.randint(2, 8)
                G.add_edge(j1, j2, capacity=cap, cost=cost, distance=distance)
                if random.random() < 0.5:
                    G.add_edge(j2, j1, capacity=cap, cost=cost, distance=distance)

    for consumer in consumers:
        for junction in random.sample(junctions, 1):
            u_x, u_y = G.nodes[junction]['x'], G.nodes[junction]['y']
            v_x, v_y = G.nodes[consumer]['x'], G.nodes[consumer]['y']
            distance = math.sqrt((u_x - v_x)**2 + (u_y - v_y)**2)
            G.add_edge(junction, consumer,
                       capacity=random.randint(100, 300),
                       cost=random.randint(3, 10),
                       distance=distance)

    nodes = []
    for n, a in G.nodes(data=True):
        d = {"id": n, "type": a["type"], "x": a["x"], "y": a["y"]}
        if "capacity" in a:
            d["capacity"] = a["capacity"]
        if "demand" in a:
            d["demand"] = a["demand"]
        nodes.append(d)

    edges = [{"source": u, "target": v, "capacity": d["capacity"],
              "cost": d["cost"], "distance": d["distance"]}
             for u, v, d in G.edges(data=True)]

    return {
        "nodes": nodes,
        "edges": edges,
        "source": reservoirs[0] if reservoirs else "",
        "sink": consumers[0] if consumers else ""
    }

def find_path(network):
    G = nx.DiGraph()
    for e in network['edges']:
        G.add_edge(e['source'], e['target'])
    try:
        return nx.shortest_path(G, source=network['source'], target=network['sink'])
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        return None

def calculate_flow(network, path):
    if not path: return 0
    min_cap = float('inf')
    for i in range(len(path) - 1):
        for e in network['edges']:
            if e['source'] == path[i] and e['target'] == path[i+1]:
                min_cap = min(min_cap, e['capacity'])
                break
    return min_cap if min_cap != float('inf') else 0

def visualize_network(network, path=None):
    fig = go.Figure()
    colors = {"source": "blue", "junction": "green", "sink": "red"}
    sizes = {"source": 10, "junction": 8, "sink": 10}

    for e in network['edges']:
        s = next(n for n in network['nodes'] if n['id'] == e['source'])
        t = next(n for n in network['nodes'] if n['id'] == e['target'])
        in_path = path and any(path[i] == e['source'] and path[i+1] == e['target'] for i in range(len(path) - 1))
        fig.add_trace(go.Scatter(
            x=[s['x'], t['x']], y=[s['y'], t['y']],
            mode='lines', line=dict(width=2 if in_path else 1, color='red' if in_path else 'gray'),
            opacity=0.9 if in_path else 0.3,
            hoverinfo='text',
            text=f"{e['source']} → {e['target']}<br>Cap: {e['capacity']}<br>Cost: {e['cost']}"
        ))

    for n in network['nodes']:
        t = n['type']
        in_path = path and n['id'] in path
        fig.add_trace(go.Scatter(
            x=[n['x']], y=[n['y']],
            mode='markers+text' if in_path or t in ['source', 'sink'] else 'markers',
            text=n['id'] if (in_path or t in ['source', 'sink']) else '',
            textposition='top center',
            marker=dict(
                size=sizes[t] * (1.8 if in_path else 1),
                color=colors[t],
                line=dict(width=1, color='black'),
                symbol='star' if in_path else 'circle'
            ),
            name=t.capitalize(),
            hovertext=f"ID: {n['id']}<br>Type: {t}" +
                      (f"<br>Cap: {n['capacity']}" if 'capacity' in n else "") +
                      (f"<br>Demand: {n['demand']}" if 'demand' in n else ""),
            hoverinfo='text',
            showlegend=t not in [trace.name for trace in fig.data]
        ))

    fig.update_layout(
        title='Water Distribution Network',
        showlegend=True, height=900, width=1200,
        xaxis=dict(title='Pressure 1', showgrid=True, range=[-5, 105]),
        yaxis=dict(title='Pressure 2', showgrid=True, range=[-5, 105]),
        plot_bgcolor='white',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig

# Load and run
with open('Filtered_Data.csv', 'r') as f:
    csv_data = f.read()

water_network = create_water_network(csv_data)
path = find_path(water_network)
flow = calculate_flow(water_network, path) if path else 0

print(f"\nNetwork Summary:")
print(f" - Reservoirs: {len([n for n in water_network['nodes'] if n['type'] == 'source'])}")
print(f" - Junctions: {len([n for n in water_network['nodes'] if n['type'] == 'junction'])}")
print(f" - Consumers: {len([n for n in water_network['nodes'] if n['type'] == 'sink'])}")
print(f" - Edges: {len(water_network['edges'])}")
print(f"\nSource: {water_network['source']}")
print(f"Sink: {water_network['sink']}")
if path:
    print(f"\nPath found ({len(path)} nodes)")
    print(f"Max flow: {flow} units")
else:
    print("No path found from source to sink!")

fig = visualize_network(water_network, path)
fig.show()



Network Summary:
 - Reservoirs: 2
 - Junctions: 10
 - Consumers: 5
 - Edges: 27

Source: reservoir_3
Sink: consumer_909
No path found from source to sink!
